**Общая информация**

**Срок сдачи:** 21 октября 2019, 08:30 

**Штраф за опоздание:** по 0.5 балла за 24 часа задержки. Через 10 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0919, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Невыполнение PEP8 -0.5 баллов
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [394]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata, fetch_20newsgroups

from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KDTree
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

In [397]:
%%pycodestyle
import collections
from scipy.spatial import distance


class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):

    def __init__(self, n_neighbors, algorithm='brute'):
        self.n = n_neighbors
        self.algorithm = algorithm
        pass

    def fit(self, X, y):
        self.x_train = X
        self.y_train = y
        if self.algorithm == 'kd_tree':
            self.tree = KDTree(X, 2)
        pass

    def predict(self, X):
        num_test = X.shape[0]
        pred = np.zeros(num_test)
        num_train = self.x_train.shape[0]
        if self.algorithm == 'brute':
            dists = distance.cdist(X, self.x_train, metric='minkowski')
            idx = np.argsort(dists, axis=1)[:, :self.n]
        elif self.algorithm == 'kd_tree':
            idx = self.tree.query(X, 2, False, False, False)
        neighbours = self.y_train[idx]
        for i in range(num_test):
            pred[i] = collections.Counter(neighbours[i]).most_common(1)[0][0]
        return pred

    def predict_proba(self, X):
        num_test = X.shape[0]
        num_train = self.x_train.shape[0]
        proba = np.zeros((num_test, len(np.unique(self.y_train))))
        if self.algorithm == 'brute':
            dists = distance.cdist(X, self.x_train, metric='minkowski')
            idx = np.argsort(dists, axis=1)[:, :self.n]
        elif self.algorithm == 'kd_tree':
            idx = self.tree.query(X, 2, False, False, False)
        neighbours = self.y_train[idx]
        for i in range(num_test):
            for j in range(self.n):
                proba[i, neighbours[i, j]] += 1
        proba /= self.n
        return proba

    def score(self, X, y):
        y_pred = self.predict(X)
        accuracy = np.mean(y.ravel() == y_pred)
        return accuracy

55:1: W391 blank line at end of file


**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [398]:
iris = datasets.load_iris()

In [399]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [400]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [401]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [402]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

In [403]:
my_clf.score(X_test, y_test)

0.9333333333333333

In [404]:
clf.score(X_test,y_test)

0.9333333333333333

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy

In [405]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.98 ms


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [406]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 38.1 µs


In [407]:
%time clf.predict(X_test)

CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 13.7 ms


array([1, 0, 0, 2, 2, 1, 1, 2, 1, 0, 1, 0, 2, 0, 1])

In [408]:
%time my_clf.predict(X_test)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 5.62 ms


array([1., 0., 0., 2., 2., 1., 1., 2., 1., 0., 1., 0., 2., 0., 1.])

In [409]:
%time clf.predict_proba(X_test)

CPU times: user 92 ms, sys: 4 ms, total: 96 ms
Wall time: 36.8 ms


array([[0. , 0.5, 0.5],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ]])

In [410]:
%time my_clf.predict_proba(X_test)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 11.7 ms


array([[0. , 0.5, 0.5],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ]])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

In [411]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [412]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [413]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 8 ms, total: 8 ms
Wall time: 26.7 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [414]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.66 ms


In [415]:
%time clf.predict(X_test)

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 26 ms


array([2, 1, 0, 1, 0, 2, 2, 2, 1, 1, 1, 0, 0, 1, 0])

In [416]:
%time my_clf.predict(X_test)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.65 ms


array([2., 1., 0., 1., 0., 2., 2., 2., 1., 1., 1., 0., 0., 1., 0.])

In [417]:
%time clf.predict_proba(X_test)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 4.09 ms


array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [418]:
%time my_clf.predict_proba(X_test)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.44 ms


array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [419]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

In [420]:
print(my_clf.score(X_test, y_test))
print(clf.score(X_test,y_test))

0.9333333333333333
0.9333333333333333


**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [ ]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [384]:
data = newsgroups['data']
target = newsgroups['target']

Переведите во всех документах все буквы в нижний регистр и замените во всех документах символы, не
являющиеся буквами и цифрами, на пробелы. Далее разбейте текста по пробельным символам на токены(термы/слова). Удалите текста, содержащие только пробелы.

In [392]:
import re

data = [re.sub('\W', ' ', s.lower()) for s in data]
data_tok = [s.split() for s in data if (not s.isspace() and s)]
#realize here
# data_tok should be a list of lists of tokens for each line in data.

In [391]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower() , map(' '.join, data_tok))), "please make sure that you lowercase the data and drop spaced texts"

Преобразуйте датасет в разреженную матрицу scipy.sparse.csr_matrix, где значение x в позиции (i, j)
означает, что в документе i слово j встретилось x раз

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальный score в среднем на валидации на 3 фолдах).
Постройте график зависимости среднего score от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf преобразование( sklearn.feature_extraction.text.TfidfTransformer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим качество(score) вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.